In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

In [6]:
DATASET_SIZE = 480
DATASET_IS_BALANCED = True

training_name = 'bert-finetune_{}k_{}_{}'.format(
    DATASET_SIZE,
    'bal' if DATASET_IS_BALANCED else 'imbal',
    datetime(2024, 1, 18).strftime('%Y-%m-%d')
)

# training_args_datetime = datetime(year=2023, month=12, day=20)          # change the date to the date of training
training_storing_folder = Path(training_name).resolve()

# model_path = Path.joinpath(
#         training_storing_folder, 
#         '{}_{}_model'.format(training_name, training_args_datetime.strftime('%Y-%m-%d')))

model_path = Path.joinpath(
        training_storing_folder, 
        '{}_model'.format(training_name))

In [7]:
model_path.name

'bert-finetune_480k_bal_2024-01-18_model'

In [8]:
# save the model and tokenizer to ONNX format

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from optimum.onnxruntime import ORTModelForSequenceClassification


tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
ort_model = ORTModelForSequenceClassification.from_pretrained(
    model_path
    ,export=True)

save_directory = Path.joinpath(training_storing_folder, model_path.name + '_onnx')

if not save_directory.exists():
    save_directory.mkdir(parents=True)

print('Saving model to {}'.format(save_directory))

tokenizer.save_pretrained(save_directory)
ort_model.save_pretrained(save_directory)

/root/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-15 11:18:30.665820: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-15 11:18:30.884834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 11:18:30.884867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 11:18:30.922787: E external/

Saving model to /root/FYP/NLP/dev-workspace/sa/bert_2023-12-13/bert-finetune_480k_bal_2024-01-18/bert-finetune_480k_bal_2024-01-18_model_onnx


In [9]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# ref: https://huggingface.co/docs/optimum/onnxruntime/usage_guides/optimization

optimizer = ORTOptimizer.from_pretrained(ort_model)
optimization_config = OptimizationConfig(\
    optimization_level=2,
    enable_transformers_specific_optimizations=True,
    optimize_for_gpu=False) 

optimizer.optimize(
    save_dir=save_directory,
    optimization_config=optimization_config
)

# it will create two files:
# model_optimized.onnx
# ort_config.json

/root/miniforge3/envs/fyp-test2-wsl/lib/python3.9/site-packages/optimum/onnxruntime/configuration.py:770: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in /root/FYP/NLP/dev-workspace/sa/bert_2023-12-13/bert-finetune_480k_bal_2024-01-18/bert-finetune_480k_bal_2024-01-18_model_onnx/ort_config.json
Optimized model saved at: /root/FYP/NLP/dev-workspace/sa/bert_2023-12-13/bert-finetune_480k_bal_2024-01-18/bert-finetune_480k_bal_2024-01-18_model_onnx (external data format: False; saved all tensor to one file: True)


PosixPath('/root/FYP/NLP/dev-workspace/sa/bert_2023-12-13/bert-finetune_480k_bal_2024-01-18/bert-finetune_480k_bal_2024-01-18_model_onnx')

In [10]:
# load the ONNX model and tokenizer

from transformers import AutoTokenizer
from onnxruntime import InferenceSession

tokenizer = AutoTokenizer.from_pretrained(save_directory)
session = InferenceSession(Path.joinpath(
    save_directory,
    "model_optimized.onnx")
)

input_names = [label.name for label in session.get_inputs()]
output_names = [label.name for label in session.get_outputs()]

# ONNX Runtime expects NumPy arrays as input
inputs = tokenizer("Using DistilBERT with ONNX Runtime!", return_tensors="np")
outputs = session.run(output_names=output_names, input_feed=dict(inputs))

In [11]:
input_names

['input_ids', 'attention_mask', 'token_type_ids']

In [12]:
output_names

['logits']

In [13]:
# load huggingface original model

from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification


hg_model = AutoModelForSequenceClassification.from_pretrained(model_path)

In [14]:
test_data = [['I like the game'], ["I do not like it."], ["It crashes when I just run on my pc."]]

In [15]:
# test inference

pred_hg = []
perd_onnx = []

for i in range(len(test_data)):

    # hg inference
    hg_inputs = tokenizer(test_data[i], return_tensors="pt", max_length=tokenizer.model_max_length, truncation=True)
    hg_outputs = hg_model(**hg_inputs)

    # onnx inference
    onnx_inputs = tokenizer(test_data[i], return_tensors="np", max_length=tokenizer.model_max_length, truncation=True)
    onnx_outputs = session.run(output_names=output_names, input_feed=dict(onnx_inputs))

    perd_onnx.append(onnx_outputs[0])   # only get the unsoftmaxed logits
    pred_hg.append(hg_outputs.logits.detach().numpy())  # only get the unsoftmaxed logits


print(pred_hg)
print(perd_onnx)

# compare the results
for i in range(len(pred_hg)):
    np.testing.assert_allclose(pred_hg[i], perd_onnx[i], rtol=1e-3, atol=1e-3)

[array([[-2.3883157,  2.4536176]], dtype=float32), array([[ 1.5583036, -2.9529977]], dtype=float32), array([[ 0.69697875, -1.636167  ]], dtype=float32)]
[array([[-2.3883147,  2.4536157]], dtype=float32), array([[ 1.5583022, -2.9529963]], dtype=float32), array([[ 0.69697833, -1.6361668 ]], dtype=float32)]
